## 1- Preprocesamiento del Texto

In [ ]:
import spacy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar el modelo de SpaCy
nlp = spacy.load("es_core_news_sm")

# Preprocesar el texto
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

# Aplicar preprocesamiento
df['COMENTARIO_PROCESADO'] = df['COMENTARIO'].apply(preprocess_text)

# Crear TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_text = vectorizer.fit_transform(df['COMENTARIO_PROCESADO']).toarray()


## 2- Etiquetado para Sentimientos y Clasificación

In [ ]:
# Análisis de sentimientos (Positivo/Negativo)
def get_sentiment(text):
    sentiment = nlp(text)._.polarity
    return 'Positivo' if sentiment > 0 else 'Negativo'

df['SENTIMIENTO'] = df['COMENTARIO_PROCESADO'].apply(get_sentiment)

# Codificar el sentimiento
le_sent = LabelEncoder()
y_sent = le_sent.fit_transform(df['SENTIMIENTO'])

# Seleccionar la etiqueta de clasificación (SEGMENTO, PAIS, PERIODO)
label_choice = input("Ingrese el label para la clasificación ('SEGMENTO', 'PAIS', 'PERIODO'): ")
y_class = df[label_choice]

# Codificar la etiqueta de clasificación
le_class = LabelEncoder()
y_class_encoded = le_class.fit_transform(y_class)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_class, y_test_class, y_train_sent, y_test_sent = train_test_split(
    X_text, y_class_encoded, y_sent, test_size=0.2, random_state=42)


## 3- Modelo Multitarea (Análisis de Sentimientos + Clasificación de texto)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definir la entrada
input_layer = layers.Input(shape=(X_train.shape[1],))

# Capa densa compartida
shared_layer = layers.Dense(128, activation='relu')(input_layer)
shared_layer = layers.Dropout(0.3)(shared_layer)

# Salida para la clasificación (Segmento, País, Periodo)
output_class = layers.Dense(len(le_class.classes_), activation='softmax', name='class_output')(shared_layer)

# Salida para el análisis de sentimientos
output_sent = layers.Dense(1, activation='sigmoid', name='sentiment_output')(shared_layer)

# Definir el modelo con dos salidas
model = models.Model(inputs=input_layer, outputs=[output_class, output_sent])

# Compilar el modelo con pérdidas separadas
model.compile(optimizer='adam',
              loss={'class_output': 'sparse_categorical_crossentropy', 'sentiment_output': 'binary_crossentropy'},
              metrics={'class_output': 'accuracy', 'sentiment_output': 'accuracy'})

# Resumen del modelo
model.summary()


## 4- Entrenamiento del Modelo

In [ ]:
# Entrenar el modelo multitarea
history = model.fit(X_train, 
                    {'class_output': y_train_class, 'sentiment_output': y_train_sent},
                    validation_data=(X_test, {'class_output': y_test_class, 'sentiment_output': y_test_sent}),
                    epochs=10,
                    batch_size=32)

# Guardar el modelo entrenado
model.save('modelo_multitarea.h5')


## 5- Evaluación del Modelo

In [ ]:
# Evaluar el modelo multitarea
loss, class_loss, sentiment_loss, class_acc, sentiment_acc = model.evaluate(
    X_test, 
    {'class_output': y_test_class, 'sentiment_output': y_test_sent})

print(f'Loss: {loss}')
print(f'Class Loss: {class_loss}, Class Accuracy: {class_acc}')
print(f'Sentiment Loss: {sentiment_loss}, Sentiment Accuracy: {sentiment_acc}')


## 6- Visualización con Tensorboard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Definir los logs de TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Volver a entrenar el modelo con TensorBoard
history = model.fit(X_train, 
                    {'class_output': y_train_class, 'sentiment_output': y_train_sent},
                    validation_data=(X_test, {'class_output': y_test_class, 'sentiment_output': y_test_sent}),
                    epochs=10,
                    batch_size=32,
                    callbacks=[tensorboard_callback])

# Para ver los resultados, ejecutar en la terminal: tensorboard --logdir logs/fit
